# Lecture 7: Integer Programming

## Contents
- [Uber POOL](#section1)
    - [TSP](#subsection1.1)
    - [TSP with precedence constraints](#subsection1.2)
- [Sudoku](#section2)
- [Rue La La](#section3)
    - [IP for a fixed total price](#subsection3.1)
    - [Finding the best total price](#subsection3.2)

## Uber POOL <a id="section1"></a>
### TSP <a id="subsection1.1"></a>
The mathematical model proposed by Miller, Tucker and Zemlin for TSP is given below:
\begin{equation}
\begin{split}
\min~ & \sum_{i=1}^n\sum_{j=1}^nc_{ij}x_{ij} \\
s.t. ~& \sum_{i=1}^nx_{ij} =1, ~ j =1,...,n\\
& \sum_{j=1}^nx_{ij} =1, ~ i =1,...,n\\
& u_i + 1 - u_j \leq M(1 - x_{ij}), ~ i\neq j, 1\leq i\leq n, 2\leq j \leq n \\ 
& x_{ij}\in\{0,1\}, u_i\geq 0, i =1,...,n.
\end{split}
\end{equation}




In [1]:
from gurobipy import *
import numpy as np

#########Parameters Set-up############

#traveling cost from node i to node j
cost = np.array([[1000, 4, 2, 1000, 1000, 1000],
                 [0, 1000, 6, 1000, 1, 7], 
                 [0, 6, 1000, 2, 1, 1000],
                 [0, 1000, 2, 1000, 1, 3],
                 [0, 1, 1, 1, 1000, 1], 
                 [0, 7, 1000, 3, 1, 1000]])

N = cost.shape[0]

#the big M
M = 10000

In [2]:
#########Model Set-up############


tsp = Model("traveling_salesman")

# Creat variables
x = tsp.addVars(N, N, vtype=GRB.BINARY, name = "x")

u = tsp.addVars(N, name = "u")

# Set objective
tsp.setObjective( quicksum(cost[i,j]*x[i,j] for i in range(N) for j in range(N)), GRB.MINIMIZE)

# Assignment constraints:
tsp.addConstrs(( quicksum(x[i,j] for j in range(N)) == 1 for i in range(N) ))
 
tsp.addConstrs(( quicksum(x[i,j] for i in range(N)) == 1 for j in range(N) ))

# Subtour-breaking constraints:
tsp.addConstrs(( u[i] + 1 - u[j] <= M*(1 - x[i,j])  for i in range(N) for j in range(1,N) ))


# Solving the model
tsp.optimize()


Academic license - for non-commercial use only
Optimize a model with 42 rows, 42 columns and 152 nonzeros
Variable types: 6 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 2010.0000000
Presolve removed 5 rows and 6 columns
Presolve time: 0.03s
Presolved: 37 rows, 36 columns, 152 nonzeros
Variable types: 5 continuous, 31 integer (31 binary)

Root relaxation: objective 9.000000e+00, 23 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.00000    0    4 2010.00000    9.00000   100%     -    0s
H    0     0                       9.0000000    9.00000  0.00%     -    0s

Explored 1 nodes (23 simplex iterations) in 0.30 seconds
Thread count was 8 (of 8 available processors

In [3]:
#  Print optimal x for x nonzero and optimal value
s_edge = []
for v in x:    
    if x[v].x > 0.001:
        print(x[v].VarName, x[v].x)
        #add both of the indicies by 1
        edge = np.add(v, (1,1))
        #append the edge to the resulting list of edges
        s_edge.append(edge)


print('\n------------------------------')
print(s_edge)
print('\n------------------------------')
for v in u: 
    print(u[v].VarName, u[v].x)

x[0,2] 1.0
x[1,0] 1.0
x[2,3] 1.0
x[3,5] 1.0
x[4,1] 1.0
x[5,4] 1.0

------------------------------
[array([1, 3]), array([2, 1]), array([3, 4]), array([4, 6]), array([5, 2]), array([6, 5])]

------------------------------
u[0] 0.0
u[1] 5.0
u[2] 1.0
u[3] 2.0
u[4] 4.0
u[5] 3.0


In [4]:
#  Obtain the permutation as a representation of the tour

permu = np.ones(N)
predecessor = 1
for i in range(N):
    for s in s_edge:
        if s[0] == predecessor:
            permu[i] = s[0]
            predecessor = s[1]
            break    
    
print(permu)

[1. 3. 4. 6. 5. 2.]


### TSP with precedence constraints <a id="subsection1.2"></a>
Suppose all the precedence pairs are summarized in the set $P$. Then a feasible tour must satisfy:
\begin{equation}
u_i\leq u_j, ~ \forall (i,j)\in P.
\end{equation}

In [5]:
# data for the precedent pair

Precedent_Pair = tuplelist([(0,2), (1,3), (4,5)])


tsp.addConstrs( (u[i] <= u[j] for (i,j) in Precedent_Pair)  )


# Solving the new model
tsp.optimize()

Optimize a model with 45 rows, 42 columns and 158 nonzeros
Variable types: 6 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]

MIP start did not produce a new incumbent solution

Found heuristic solution: objective 1016.0000000
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolved: 39 rows, 36 columns, 152 nonzeros
Variable types: 5 continuous, 31 integer (31 binary)

Root relaxation: objective 9.000200e+00, 24 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.00020    0    6 1016.00000    9.00020  99.1%     -    0s
H    0     0                      15.0000000    9.00020  40.0%     -    0s
H    0     0                      11.0000000    9.00020  18.2%     -    0s
     0     0   10.0000

In [6]:
#  The list of edges traversed
s_edge = []
for v in x:    
    if x[v].x > 0.001:
        edge = np.add(v, (1,1))
        s_edge.append(edge)
        
#  Obtain the permutation as a representation of the tour
permu = np.ones(N)
predecessor = 1
for i in range(N):
    for s in s_edge:
        if s[0] == predecessor:
            permu[i] = s[0]
            predecessor = s[1]
            break    
    
print(permu)

[1. 2. 5. 3. 4. 6.]


## Sudoku <a id="section2"></a>
Integer programming formulation for the Sudoku problem:
\begin{equation}
\begin{split}
\min~ & 0 \\
s.t. ~& x_{ijy_{ij}} =1, ~ y_{ij}\neq 0\\
& \sum_{i=1}^9x_{ijk} = 1, ~ j,k =1,...,9\\
& \sum_{j=1}^9x_{ijk} = 1, ~ i,k =1,...,9\\
& \sum_{k=1}^9x_{ijk} = 1, ~ i,j =1,...,9\\
& \sum_{i=3p-2}^{3p}\sum_{j=3q-2}^{3q}x_{ijk} = 1, ~ k=1,...,9, p,q =1,2,3\\
& x_{ijk}\in\{0,1\}.
\end{split}
\end{equation}

In [7]:

#########Parameters Set-up############

y = np.array([[8, 0, 0, 6, 0, 0, 9, 0, 5],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 2, 0, 3, 1, 0],
              [0, 0, 7, 3, 1, 8, 0, 6, 0],
              [2, 4, 0, 0, 0, 0, 0, 7, 3],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 2, 7, 9, 0, 1, 0, 0],
              [5, 0, 0, 0, 8, 0, 0, 3, 6], 
              [0, 0, 3, 0, 0, 0, 0, 0, 0]])

print(y)

N = y.shape[0]


[[8 0 0 6 0 0 9 0 5]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 3 1 0]
 [0 0 7 3 1 8 0 6 0]
 [2 4 0 0 0 0 0 7 3]
 [0 0 0 0 0 0 0 0 0]
 [0 0 2 7 9 0 1 0 0]
 [5 0 0 0 8 0 0 3 6]
 [0 0 3 0 0 0 0 0 0]]


In [8]:
#########Model Set-up############

m = Model("Sudoku")

# Creat variables
x = m.addVars(N, N, N, vtype=GRB.BINARY, name = "x")

# Set objective
m.setObjective(0, GRB.MINIMIZE)

# Fill-in constraints:
for i in range(N):
    for j in range(N):
        if y[i,j] > 0:
            m.addConstr( x[i,j, y[i,j]-1] == 1 )

# For every column, each digit appears only once:
m.addConstrs(( quicksum(x[i,j,k] for i in range(N)) == 1  for j in range(N) for k in range(N) ))

# For every row, each digit appears only once:
m.addConstrs(( quicksum(x[i,j,k] for j in range(N)) == 1  for i in range(N) for k in range(N) ))

# For every entry (i,j), only one digit is chosen:
m.addConstrs(( quicksum(x[i,j,k] for k in range(N)) == 1  for i in range(N) for j in range(N) ))

# For 3x3 square, each digit appear only once
m.addConstrs(( quicksum(x[i,j,k] for i in range(3*p, 3*(p+1)) for j in range(3*q, 3*(q+1))) == 1
              for k in range(N) for p in range(3) for q in range(3) ))



# Solving the model
m.optimize()


Optimize a model with 349 rows, 729 columns and 2941 nonzeros
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 345 rows and 723 columns
Presolve time: 0.12s
Presolved: 4 rows, 6 columns, 14 nonzeros
Variable types: 0 continuous, 6 integer (6 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.17 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [9]:
# Print out the solution to the Sudoku problem
for i in range(N):
    for j in range(N):
        for k in range(N):
            if x[i,j,k].x == 1:
                print("%3i" % (k+1), end =" ") 
    print("\n")
    

  8   1   4   6   3   7   9   2   5 

  3   2   5   1   4   9   6   8   7 

  7   9   6   8   2   5   3   1   4 

  9   5   7   3   1   8   4   6   2 

  2   4   1   9   5   6   8   7   3 

  6   3   8   2   7   4   5   9   1 

  4   6   2   7   9   3   1   5   8 

  5   7   9   4   8   1   2   3   6 

  1   8   3   5   6   2   7   4   9 



## Ruelala <a id="section3"></a>
### Integer programming formulation for a fixed total price $k$<a id="subsection3.1"></a>

\begin{equation}
\begin{split}
z_k = \max~ & \sum_{n=1}^N\sum_{m=1}^Mp_m D(n,m,k)x_{n,m}\\
s.t. ~& \sum_{m=1}^Mx_{n,m} = 1, ~ n=1,...,N\\
& \sum_{n=1}^N\sum_{m=1}^Mp_mx_{n,m} = k,\\
& x_{n,m}\in\{0,1\}.
\end{split}
\end{equation}
Here, $n=1,...,N$ is the index for the products and $m=1,...,M$ is the index for the price point.

In [10]:
#########Parameters Set-up############
# For convenience demands are put 0 for impossible scenarios, for example, 
# the price for product is 39.99 but the total prices is 3x59.99, i.e., the cell demand[0,0,12] below.


demand = np.array([   [[30, 32, 33, 35, 38, 40, 44, 50, 50,  0,  0,  0,  0],
                       [ 0, 29, 31, 32, 30, 34, 38, 40, 42, 44,  0,  0,  0],
                       [ 0,  0, 25, 29, 28, 28, 31, 33, 35, 36, 38,  0,  0],
                       [ 0,  0,  0, 10, 18, 18, 20, 21, 24, 26, 26, 27,  0],
                       [ 0,  0,  0,  0,  2,  4,  4,  6,  8, 10, 12, 15, 16]],
                   
                      [[60, 65, 68, 70, 73, 76, 78, 82, 83,  0,  0,  0,  0],
                       [ 0, 50, 52, 53, 55, 57, 59, 60, 64, 65,  0,  0,  0],
                       [ 0,  0, 28, 35, 37, 40, 42, 43, 43, 44, 45,  0,  0],
                       [ 0,  0,  0,  7,  9,  9, 10, 12, 12, 14, 14, 14,  0],
                       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  2,  2,  2]],
                   
                      [[20, 20, 20, 21, 21, 22, 22, 24, 24,  0,  0,  0,  0],
                       [ 0, 20, 20, 20, 21, 21, 21, 22, 22, 22,  0,  0,  0],
                       [ 0,  0, 19, 19, 20, 20, 21, 22, 22, 22, 23,  0,  0],
                       [ 0,  0,  0, 17, 18, 18, 20, 20, 20, 20, 20, 20,  0],
                       [ 0,  0,  0,  0, 15, 15, 15, 16, 18, 16, 17, 17, 18]]      ])

# number of product, number of price points, number of total price points
N, M, K = demand.shape

# minimum and maximum price point
min_price = 39.99
max_price = 59.99

# vector of prices
price_v = np.linspace(min_price, max_price, num = M)

# vector of all possible total prices
total_price_v = np.linspace(N*min_price, N*max_price, num = K)

# initialize the index of the total price as the mimnimum one
k = 0

print(price_v)
print(total_price_v)

[39.99 44.99 49.99 54.99 59.99]
[119.97 124.97 129.97 134.97 139.97 144.97 149.97 154.97 159.97 164.97
 169.97 174.97 179.97]


In [11]:
#########Model Set-up############

def model_setup():
    
    m = Model("Ruelala")

    # Creat variables
    x = m.addVars(N, M, vtype=GRB.BINARY, name = "x")
    
    # set objective
    m.setObjective( quicksum(price_v[m]*demand[n,m,k]*x[n,m] for n in range(N) for m in range(M)), GRB.MAXIMIZE)
    
    # for each product, only one price point can be chosen:
    m.addConstrs( ( quicksum(x[n,m] for m in range(M)) == 1 for n in range(N))  )

    # total price constraint:
    m.addConstr( quicksum(price_v[m]*x[n,m] for n in range(N) for m in range(M)) == total_price_v[k]  ) 
    
    #Supressing the optimization output
    m.setParam( 'OutputFlag', False )
    
    return m


### Finding the best total price $k$<a id="subsection3.2"></a>
For each $k=1,...,K$, we solve the above problem and obtain $z_k$. We then take the maximium:
$$
\max_{k=1,...,K}z_k.
$$

In [12]:
# initialize the vector of profits
profit_v = np.zeros(K)

# initialize the vector of optimal prices
opt_price_v = np.zeros( (K, N) )

for k in range(K):
    
    # setup the model 
    m_rll = model_setup()

    # solving the model
    m_rll.optimize()
    
    # storing the corresponding information
    profit_v[k] = m_rll.objVal
    
    # extract the variables from the model. NOTE: variables extracted in this way are automatically formatted as a vector
    x = m_rll.getVars()
    
    # reformat the vector as a matrix with dimension NxM
    x = np.reshape(x, (N,M))
    
    for n in range(N):
        for m in range(M):
            if x[n,m].x == 1:
                opt_price_v[k,n] = price_v[m]
    

# find the total price that maximizes the profit

k_max = np.argmax(profit_v)

print("The maximum profit is: %g" % profit_v[k_max])

print("The optimal price is: ", opt_price_v[k_max,:])


The maximum profit is: 5568.76
The optimal price is:  [49.99 39.99 59.99]


In [13]:
print(profit_v)

[4398.9  4778.83 5013.81 5188.79 5338.74 5468.75 5568.76 5393.81 5278.94
 4589.14 3989.26 2689.54 2159.64]


In [14]:
print(opt_price_v)

[[39.99 39.99 39.99]
 [39.99 39.99 44.99]
 [44.99 39.99 44.99]
 [44.99 39.99 49.99]
 [39.99 39.99 59.99]
 [44.99 39.99 59.99]
 [49.99 39.99 59.99]
 [54.99 39.99 59.99]
 [54.99 44.99 59.99]
 [54.99 49.99 59.99]
 [59.99 49.99 59.99]
 [59.99 54.99 59.99]
 [59.99 59.99 59.99]]
